# Desenvolvimento do Modelo LSTM

Este notebook implementa um modelo de deep learning utilizando LSTM para capturar padrões temporais nos dados de preços das ações baixadas no arquivo `PETR4.SA_data.csv`.

In [ ]:
# Instalar bibliotecas necessárias
%pip install pandas numpy scikit-learn tensorflow

Note: you may need to restart the kernel to use updated packages.


In [8]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [10]:
# Carregar os dados
data = pd.read_csv('../downloadData/data/PETR4.SA_data.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Ajustar as colunas relacionadas à moeda para duas casas decimais
for col in ['Close', 'High', 'Low', 'Open']:
    data[col] = data[col].round(2)

# Garantir que a coluna Volume seja tratada como número inteiro
data['Volume'] = data['Volume'].astype(int)

# Selecionar a coluna de fechamento ajustado
features = data[['Close', 'Open', 'High', 'Low', 'Volume']]
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)
# X, y = create_sequences(scaled_features, sequence_length)

In [ ]:
# Normalizar os dados
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled_prices = scaler.fit_transform(prices)

In [11]:
# Criar sequências para o modelo LSTM
def create_sequences(data, sequence_length):
    sequences = []
    labels = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        labels.append(data[i + sequence_length][0])  # Apenas o índice 0 (Close)
    return np.array(sequences), np.array(labels)

sequence_length = 60
X, y = create_sequences(scaled_features, sequence_length)

In [12]:
# Dividir os dados em conjuntos de treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Construir o modelo LSTM
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(50, return_sequences=False),
    Dense(25),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

/home/diogo/Fase 4/TechChallenge_Fase4/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
# Treinar o modelo
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32)

Epoch 1/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 0.0127 - val_loss: 2.5940e-04
Epoch 2/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 2.6935e-04 - val_loss: 2.3612e-04
Epoch 3/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 2.7031e-04 - val_loss: 2.2683e-04
Epoch 4/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 2.2517e-04 - val_loss: 2.2385e-04
Epoch 5/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 2.5062e-04 - val_loss: 1.7882e-04
Epoch 6/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 1.9426e-04 - val_loss: 1.7742e-04
Epoch 7/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 1.8006e-04 - val_loss: 3.6344e-04
Epoch 8/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 2.1307e-04 - val_loss: 1.3446e-04
Epoch 9/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 1.7292e-04 - val_loss: 1.1373e-04
Epoch 10/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 1.4519e-04 - val_loss: 1.4931e-04
Epoch 11/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step

In [15]:
# Avaliar o modelo
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

y_pred = model.predict(X_val)
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
mape = mean_absolute_percentage_error(y_val, y_pred)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'MAPE: {mape}')

40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
MAE: 0.004797273147369932
RMSE: 0.007820802463986322
MAPE: 0.04804133034711952


In [18]:
# Salvar o modelo treinado
model.save('../app/model/model_lstm.keras')
print('Modelo salvo com sucesso!')

Modelo salvo com sucesso!
